# IMDB- Part 2

- stakeholder wants to extract the budget, revenue, and MPAA Rating (G/PG/PG-13/R), which is also called "Certification".

## Specifications
Financial Data

- perform a test extraction of movies that started in 2000 or 2001

- Each year should be saved as a separate .csv.gz file

- One function will add the certification (MPGG Rating) to movie.info

- The other function will help you append/extend a JSON file with Python

- Once you have retrieved and saved the final results to 2 separate .csv.gz files, move on to a new Exploratory Data Analysis notebook to explore the following questions.

## Exploratory Data Analysis

- Load in your csv.gz's of results for each year extracted.
- Concatenate the data into 1 dataframe for the remainder of the analysis.
- Once you have your data from the API, they would like you to perform some light EDA to show:
 - How many movies had at least some valid financial information (values > 0 for budget OR revenue)?
 - Please exclude any movies with 0's for budget AND revenue from the remaining visualizations.
 - How many movies are there in each of the certification categories (G/PG/PG-13/R)?
 - What is the average revenue per certification category?
 - What is the average budget per certification category?


## Import libraries

In [1]:
# Standard Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports

import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook
import tmdbsimple as tmdb

In [2]:
#json and loading creds

with open('/Users/alicia/.secret/tmbd_api.json', 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
#creating variable for api-key

tmdb.API_KEY = login['api-key']

In [4]:
#defining movie

movie = tmdb.Movies(603)
movie

In [5]:
#check movie info, doesn't have certification so will add this first

info = movie.info()
info

{'adult': False,
 'backdrop_path': '/ncEsesgOJDNrTUED89hYbA117wo.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 62.542,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 372,
   'logo_path': None,
   'name': 'Groucho II Film

In [6]:
#Function to add movie certification

def get_movie_with_rating(movie_id):
    # Getting movie object for current id
    movie = tmdb.Movies(movie_id)
    # Saving .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    # Looping through the countries in releases
    for c in releases['countries']:
        # If the country abbreviation == US
        if c['iso_3166_1'] == 'US':
            # Saving a certification key in the info dictionaries with the certification
            info['certification'] = c['certification']
    return info


In [7]:
#test function to make sure works; cert is there
get_movie_with_rating(300)

{'adult': False,
 'backdrop_path': '/bHePzkyRcMhnab2qZbhj1bCElnf.jpg',
 'belongs_to_collection': None,
 'budget': 6000000,
 'genres': [{'id': 35, 'name': 'Comedy'},
  {'id': 18, 'name': 'Drama'},
  {'id': 14, 'name': 'Fantasy'}],
 'homepage': 'http://www.lasciencedesreves-lefilm.com/accueil.htm',
 'id': 300,
 'imdb_id': 'tt0354899',
 'original_language': 'fr',
 'original_title': 'La Science des rêves',
 'overview': 'A man entranced by his dreams and imagination is lovestruck with a French woman and feels he can show her his world.',
 'popularity': 6.814,
 'poster_path': '/1qCq228LsNtUseCnNE7Naw6NBUz.jpg',
 'production_companies': [{'id': 11911,
   'logo_path': None,
   'name': 'Partizan Films',
   'origin_country': ''},
  {'id': 9,
   'logo_path': '/nda3dTUYdDrJ6rZqBpYvY865aDv.png',
   'name': 'Gaumont',
   'origin_country': 'FR'},
  {'id': 591,
   'logo_path': '/q5I5RDwMEiqoNmfaJgd2LraEOJY.png',
   'name': 'France 3 Cinéma',
   'origin_country': 'FR'},
  {'id': 6586, 'logo_path': None

## Before Loops

In [8]:
#defining folder where files will go after

FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['tmdb_api_results_2000.json',
 'final_tmdb_data_2000.csv.gz',
 'tmdb_api_results_2001.json',
 'title_basics.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz',
 'title_ratings.csv.gz']

In [9]:
# basics needs to be defined

basics = pd.read_csv('/Users/alicia/Documents/DataEnrichment/IMDB Movies Project/IMDB-Production-Success-Analysis/Data/title_basics.csv.gz')
basics.head() 

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0093119,movie,Grizzly II: Revenge,Grizzly II: The Predator,0,2020,NaN,74,"Horror,Music,Thriller"


In [10]:
#defining years that stakeholder watns

years_to_get = [2000,2001]

## Outer & Inner Loop

In [11]:
# Beginning outer loop
for YEAR in tqdm_notebook(years_to_get, desc='YEARS',position=0):
    # Dsignating JSON file to store results
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # Checking if file exists
    file_exists = os.path.isfile(JSON_FILE)

    # Creating file in case it doesn't exist
    if file_exists == False:
        # Saving empty dict w/ imdb_id to new json file
        with open(JSON_FILE, 'w') as f:
             json.dump([{'imdb_id':0}],f)

    # Saving new year as the current df
    df = basics.loc[basics['startYear']==YEAR].copy()
    # Saving movie id's to list
    movie_ids = df['tconst'].copy()

    # Loading existing data from json into a dataframe
    prev_df = pd.read_json(JSON_FILE)
    prev_df

    # Filtering out id's that are already in JSON_FILE
    get_movieid = movie_ids[~movie_ids.isin(prev_df['imdb_id'])]

    # Defining function to be used in the loop
    def write_json(new_data, filename):
        with open(filename, 'r+') as file:
            # Loading existing data into a dict
            file_data = json.load(file)
            # Append or extend
            if (type(new_data) == list) & (type(file_data) == list):
                file_data.extend(new_data)
            else:
                file_data.append(new_data)
            # Setting files current position at offset
            file.seek(0)
            # Converting back to .json
            json.dump(file_data, file)

    #Getting index and movie id from list
    for movie_id in tqdm_notebook(get_movieid,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempting to retrieve data for movie_id
        try:
            # Using premade movie_id function
            temp = get_movie_with_rating(movie_id) 
            # Applying premade write_json function to appen/extend
            write_json(temp,JSON_FILE)
            # Applying a short 20ms sleep to prevent overwhelming server
            time.sleep(0.02)

        # If it fails, make a dict with just the id and None for certification.
        except Exception as e:
            continue

    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/209 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/241 [00:00<?, ?it/s]